In [1]:
import numpy as np
import random
import statistics as st
import scipy.stats
import time
import math

In [2]:
# Global Variables

CS = 15 #No. of charging stations

# define time slots
H = 10

# define 'tau' allowable time slots within a particular charging station
tau = 3

N = 5000 #Total number of MC simulations

Budget = 50 #Budget

# Initialize (mu, sigma) values for each CS
min_mean = 3
max_mean = 20
min_sd = 1
max_sd = 6

In [3]:
def normal_parameters():
# Function to assign the mean and standard deviation parameters
    param=[]
    
    for i in range(H):
        mean=random.randint(min_mean,max_mean)
        sd=random.randint(min_sd,max_sd)
        
        while((mean-3*sd)<0):
            sd=sd-1
            
#         B=B+mean
        param.append([mean,sd])
#     print("Probability Distributions",param) 
    return param

In [4]:
def calculateAvgDemand(parameters):
    totalMean = sum([mu[0] for mu in parameters])
    totalSD = math.sqrt(sum([sd[1]**2 for sd in parameters]))
    return [totalMean/H, totalSD/H]

In [5]:
def pdf(parameters, ports):
    mean = parameters[0]
    sd = parameters[1]
    return(scipy.stats.norm(mean,sd).pdf(ports))

def cdf(parameters, ports):
    mean = parameters[0]
    sd = parameters[1]
    return(scipy.stats.norm(mean,sd).cdf(ports))

def subtract_constant_from_ND(W, ports):
    mean = W[0]
    sd = W[1]
    
    # Calculate the Mean
    alpha = (ports - mean)/sd
    beta = (500 - mean)/sd
#     mean_new = mean - sd * ((pdf([0, 1], beta) - pdf([0, 1], alpha))/(cdf([0, 1], beta) - 
# cdf([0, 1], alpha)))
    mean_new = mean - ports

    # Calculate the Standard Deviation through truncation
    t1 = (beta * pdf([0,1],beta) - alpha * pdf([0,1],alpha) + 0.0000001)/(cdf([0,1],beta) - 
                                                                          cdf([0,1],alpha) + 0.000001)
    t2 = (pdf([0,1],beta) - pdf([0,1],alpha)+0.0000001)/(cdf([0,1],beta) - cdf([0,1],alpha)+0.000001)
    sd_new = math.sqrt(sd**2 * (1 - t1 - t2**2))
    
    return [mean_new, sd_new]

In [6]:
def calculateWaitingTimeAtATimeSlot(W):
    sumOfWT = 0
    for k in range(len(W)):
        sumOfWT = sumOfWT + W[k] * (tau - k)
    return sumOfWT

In [7]:
def generateDemands(demand):
    parameters = demand
    mean=parameters[0]
    sd=parameters[1]
    d=abs(np.random.normal(mean,sd,1))
    d=list(d.astype(int))
    d=d[0]
    return d

In [8]:
def calculateW_new(W_old, demand, ports):
    
    W_new = np.zeros(tau, dtype='int') # Saves the W(h+1) values
    
    if tau>1:
        for k in range(tau):

            demandsArrivedBeforek = 0

            W_new[0] = max(0, W_old[1]-ports)

            if k==tau-1:
                d = generateDemands(demand) #Change the time slots here

    #             Calculate the demands that have already arrived till k
                for j in range(1,k+1):
                    demandsArrivedBeforek = demandsArrivedBeforek + W_old[j]

                W_new[k] = max(0, d + min(0, demandsArrivedBeforek-ports))
#                 print("d:",d)

            else:
    #             Calculate the demands that have already arrived till k
                for j in range(1,k+1):
                    demandsArrivedBeforek = demandsArrivedBeforek + W_old[j]

                W_new[k] = max(0, W_old[k+1] + min(0, demandsArrivedBeforek-ports))

    elif tau == 1:
        d = generateDemands(demand)
        
        W_new[0] = max(0, d-ports)
#     print("Final W_new:", W_new)
    return W_new, d

In [9]:
# This method calculates the average waiting time and the EVs leaving without charge at a single CS 'c'
# Contains two sub functions:
# One, that calculates the value of W for every time slot h
# Two, that calculates the total waiting time at time slot h

def calculateWL(W_old, normal_distributions, ports):
#     print("\n")
#   'W_old' saves the W(h) values

    L = 0 # Saves the number of EVs left without getting charge
#     AvgWTfor_c = 0 # Saves the average waiting time of a charging station over its total time horizon
    L_each_h = []
    total_demand = 0
    
    for h in range(H):
        demand = normal_distributions[h]
    
        W_old, d = calculateW_new(W_old, demand, ports)
        L = L + W_old[0]
        total_demand = total_demand + d
        
        L_each_h.append(W_old[0])
        
#         Call the calculateWaitingTimeAtATimeSlot() method
#         WTat_h = calculateWaitingTimeAtATimeSlot(W_old)
#         AvgWTfor_c = AvgWTfor_c + calculateWaitingTimeAtATimeSlot(W_old)
        
#         print("time slot:", h,", param:",demand, ", W(",h,"):",W_old)
#     AvgWTfor_c = AvgWTfor_c/H
#     print("left:",L)
#     print("left/demand:",L/total_demand)
#     print(L_each_h)
    
    return L

In [10]:
def MonteCarlo(parameters, ports):
    
    L_c_store = []
    
#     Value to save the total EVs leaving without charging after each MC simulation
    Lfor_c_afterMCsim = 0 

    for sim in range(N):

    #     Initialise W to all zeros
        W_old = np.zeros(tau, dtype='int')

    #     Update in the new value 'W_new'
        L_sim = calculateWL(W_old, parameters, ports)
        
        L_c_store.append(L_sim) #All leaves after each simulation N
        
#         Add the value of simulations
        Lfor_c_afterMCsim = Lfor_c_afterMCsim + L_sim

    Lfor_c_mean = Lfor_c_afterMCsim/N

    Lfor_c_sd = st.stdev(L_c_store)
    
    return [Lfor_c_mean, Lfor_c_sd]

In [11]:
def initialAllocation(choice, normal_parameters):
    if choice == 0:
        #     Based on minimum demands at a PCS
        A = []
        B = Budget
        for p in normal_parameters:
            k = min([i[0] for i in p])
    #         print("Budget:",B,"k:",k)
            if k <= B:
                A.append(k)
                B = B - k
            else:
    #             print("Remaining Budget:",B)
                A.append(B)
                B = 0
    #     A[-1] = A[-1] + B

        ix = random.randrange(len(A)-1)
        A[ix] = A[ix] + B

#         print("Initial Allocation:",A)#, ",remaining ports:",B)
        return A
    
    elif choice == 1:
    #     Based on average total demands
        A_t = [0 for _ in range(CS)]
        B = Budget
#         print("CS:", CS, "tau:", tau, "H:", H, "Budget:",B)
        avg_demands = []
        for p in normal_parameters:
            avg = int(sum([i[0] for i in p])/H)
    #         print("average demand:", avg)
            avg_demands.append(avg)
        a = np.array(avg_demands)    
        sort = np.sort(a)
    #     print("Sorted array:", sort)
        sort_idx = np.argsort(a)
    #     print("Sorted arguments", sort_idx)
        for i in range(CS):
            k = sort[i]
    #         print("i:", i, "k:", k)
            if k <= B:
                A_t[sort_idx[i]]=k
                B = B-k
            else:
    #             print("A_t:", A_t)
    #             print("Remaining Budget:",B)
                break
    #             A_t.append(B)
    #             B = 0

        if any(ele==0 for ele in A_t):
            A_t[A_t.index(0)]=B
            B = 0

#         print("A_t:", A_t)
#         print("Remaining Budget:",B)
        while B>0:
            indiv_B = int(B/CS)
            if indiv_B > 0:
                A_t = [j+indiv_B for j in A_t]
                B = B - CS*indiv_B
            else:
                ix = random.randrange(len(A_t)-1)
                A_t[ix] = A_t[ix] + B
                B = 0
                
#             print("Remaining Budget:",B)
        return A_t
    
    elif choice == 2:
    #     Based on Average of window demands
        A_w = [0 for _ in range(CS)]
        B = Budget
        window_demands = []
        for p in normal_parameters:
            maxValue = 0
            for sec in range(H-tau+1):
                s = 0
                for t in range(tau):
                    s = s + p[sec+t][0]
                avg = int(s/tau)
                if maxValue < avg:
                    maxValue = avg
            window_demands.append(maxValue)

        a = np.array(window_demands)    
        sort = np.sort(a)
    #     print("Sorted array:", sort)
        sort_idx = np.argsort(a)
    #     print("Sorted arguments", sort_idx)
        for i in range(CS):
            k = sort[i]
    #         print("i:", i, "k:", k)
            if k <= B:
                A_w[sort_idx[i]]=k
                B = B-k
            else:
    #             print("A_t:", A_t)
    #             print("Remaining Budget:",B)
                break
    #             A_t.append(B)
    #             B = 0

        if any(ele==0 for ele in A_w):
            A_w[A_w.index(0)]=B
            B = 0

#         print("A_w:", A_w)
#         print("Remaining Budget:",B)
        return A_w

In [12]:
def greedy_distribution(A, normal_parameters):
    
    start_time=time.process_time()
    
    minMaxL = 9999
#     globalMaxL = 0
    MaxL_mean = 0   
    MinL_mean = 9999  
    sum_maxL = 0 
    
    for i in range(CS): #Calculate for each candidate charging station of initial allocation

        ports = A[i]
#         print("\nCharging Station:",i,"Ports:",ports)
            
        parameters = normal_parameters[i]
#         print("parameters:",parameters)

        leaves_i_tot = MonteCarlo(parameters, ports)
        leaves_mean = leaves_i_tot[0]
        leaves_sd = leaves_i_tot[1]
        
        sum_maxL = sum_maxL + leaves_mean# Total leaves per CS

#         MaxL saves the Max leaves of the entire allocation
        if MaxL_mean < leaves_mean:
            MaxL_mean = leaves_mean
            MaxL_sd = leaves_sd
            idx_max = i

#         MinL saves the Min leaves of the entire allocation
        if MinL_mean > leaves_mean:
            MinL_mean = leaves_mean
            MinL_sd = leaves_sd
            idx_min = i
          
    sum_maxL = sum_maxL/H      
            
    globalMaxL_prev = 9999
    Z1 = MaxL_mean
    Z2 = sum_maxL
        
    # print("\nInitial Allocation Values:")
    # print("Z1:", Z1,"\tZ2:", Z2)

    count = 0
    iteration = 1
    
#     while(globalMaxL_prev >= globalMaxL_curr):
    while(count<3):
        
        MaxL_mean = 0   
        MinL_mean = 9999   
        sum_maxL = 0
        
        A[idx_max] = A[idx_max] + 1
        A[idx_min] = A[idx_min] - 1
        
#         print("\nIter:", iteration, "Current Allocation:",A)
        
        for i in range(CS):
            ports = A[i]
#             print("\nCharging Station:",c,"Ports:",ports)
            
            parameters = normal_parameters[i]
#         print("parameters:",parameters)

            leaves_i_tot = MonteCarlo(parameters, ports)
            leaves_mean = leaves_i_tot[0]
            leaves_sd = leaves_i_tot[1]

#         MaxL saves the Max leaves of the entire allocation
            if MaxL_mean < leaves_mean:
                MaxL_mean = leaves_mean # Calculate Z1
                MaxL_sd = leaves_sd
                idx_max = i
             
            # Calculate Z2
            sum_maxL = sum_maxL + leaves_mean # Total leaves per CS

#         MinL saves the Min leaves of the entire allocation
            if MinL_mean > leaves_mean:
                MinL_mean = leaves_mean
                MinL_sd = leaves_sd
                idx_min = i
#             print("MaxL_mean:", MaxL_mean)
          
        sum_maxL = sum_maxL/H
        
        globalMaxL_prev = Z1
        Z1 = MaxL_mean
        Z2 = sum_maxL
        
#         print("Z1:", Z1)
#         print("Z2:", Z2)
        
        if (globalMaxL_prev <= Z1):
#             print("count++")
            count = count + 1
#         if count > 20:
#             break
        
        iteration += 1
    
#         print("count:", count)
    endtime=time.process_time() - start_time
    print("\nFinal Allocation Values")
    print("Z1:", Z1, ", \tZ2: (", Z2,",", MaxL_sd,") ")
    # print("\nMaxL_sd:",MaxL_sd, "\n\nFinal Allocation:", A, "\n\nTotal Iterations:", iteration)

    print("\nTotal time taken:",endtime, "secs,",endtime/60,"mins")
    
    return A
        
# greedy_distribution()

In [13]:
def main():
    
    print("********Greedy + MonteCarlo********")
    
    # Test Case I
    normal_parameters=np.array([[[13,1],[17,2],[12,3],[8,2],[5,1],[3,1],[10,3],[15,4],[8,2],[6,2]],
        [[16, 2], [4, 1], [10, 1], [4, 1], [4, 1], [16, 5], [19, 5], [4, 1], [14, 2], [7, 2]],
        [[6, 1], [4, 1], [15, 5], [9, 3], [17, 2], [19, 2], [6, 2], [14, 1], [4, 1], [15, 5]],
        [[18, 3], [5, 1], [6, 2], [12, 2], [16, 3], [16, 5], [11, 1], [14, 4], [9, 1], [5, 1]], 
        [[8, 2], [13, 3], [6, 2], [11, 2], [11, 3], [18, 1], [10, 1], [14, 4], [18, 1], [17, 1]],
                                
        [[7, 2], [11, 1], [3, 1], [19, 6], [8, 2], [5, 1], [14, 4], [4, 1], [11, 1], [20, 1]],
        [[14, 2], [4, 1], [3, 1], [8, 2], [7, 2], [5, 1], [7, 2], [4, 1], [3, 1], [20, 1]],
        [[15, 5], [10, 3], [20, 2], [10, 2], [7, 1], [8, 2], [23, 3], [7, 2], [14, 4], [13, 4]],
        [[13, 4], [22, 7], [20, 6], [17, 2], [7, 2], [18, 4], [16, 5], [23, 1], [9, 1], [19, 4]],
        [[8, 3], [5, 1], [6, 2], [6, 2], [6, 3], [6, 5], [3, 1], [4, 4], [8, 1], [5, 1]],
                                
        [[20, 4], [6, 2], [5, 1], [18, 1], [11, 3], [4, 1], [20, 2], [3, 1], [11, 3], [19, 1]],
        [[10, 3], [6, 2], [4, 1], [20, 3], [6, 2], [4, 1], [19, 4], [9, 3], [20, 3], [10, 3]],
        [[17, 1], [16, 1], [5, 1], [9, 2], [14, 4], [17, 3], [12, 1], [19, 2], [6, 2], [5, 1]],
        [[18, 1], [5, 1], [15, 2], [16, 2], [14, 3], [11, 3], [20, 4], [13, 4], [4, 1], [12, 4]],
        [[6, 2], [15, 5], [3, 1], [3, 1], [8, 2], [6, 1], [14, 4], [4, 1], [6, 2], [12, 1]]])
                                
#         [[12, 3], [17, 1], [18, 1], [20, 3], [12, 2], [19, 4], [7, 1], [20, 6], [11, 3], [10, 3]],
#         [[11, 3], [4, 1], [20, 6], [12, 3], [19, 6], [12, 4], [3, 1], [17, 2], [5, 1], [11, 3]],
#         [[10, 2], [14, 1], [17, 1], [8, 2], [18, 1], [3, 1], [9, 3], [19, 2], [4, 1], [13, 1]],
#         [[5, 1], [15, 5], [11, 1], [3, 1], [7, 2], [6, 2], [6, 2], [15, 5], [14, 2], [16, 3]],
#         [[4, 1], [5, 1], [19, 5], [10, 1], [17, 4], [12, 2], [8, 1], [12, 4], [20, 5], [8, 1]]])
                                
#         [[5, 1], [7, 2], [11, 3], [14, 4], [11, 1], [4, 1], [6, 2], [3, 1], [5, 1], [14, 1]],
#         [[18, 2], [17, 4], [10, 3], [8, 2], [16, 5], [7, 2], [19, 5], [12, 1], [14, 3], [15, 3]],
#         [[7, 2], [10, 3], [10, 2], [20, 2], [17, 4], [8, 2], [7, 2], [9, 3], [5, 1], [7, 1]],
#         [[16, 5], [20, 5], [4, 1], [9, 3], [16, 1], [9, 1], [5, 1], [3, 1], [5, 1], [14, 3]],
#         [[20, 2], [10, 3], [9, 2], [6, 2], [9, 3], [4, 1], [10, 2], [20, 5], [5, 1], [3, 1]]])


    # Test Case II
    normal_parameters = np.array([[[12, 6], [8, 2], [14, 4], [15, 4], [16, 4], [11, 6], [10, 3],[11, 6], [16, 4], 
                                   [14, 4]], 
                                  [[13, 6], [6, 2], [9, 3], [15, 6], [20, 5], [16, 5], [6, 2], [3, 6], 
                                [11, 3], [17, 5]], 
                                  [[17, 5], [18, 5], [15, 4], [13, 4], [6, 6], [4, 5], [11, 5], 
                                [10, 3], [8, 2], [20, 5]], 
                                  [[7, 5], [17, 5], [10, 4], [9, 3], [4, 1], [6, 2], [8, 4], [9, 5], [14, 4], 
                                   [18, 5]], 
                                  [[13, 4], [12, 4], [20, 5], [16, 4], [15, 4], [12, 5], [10, 6], [11, 6], 
                                   [12, 3], [6, 4]], 
                                  [[13, 4], [10, 4], [4, 5], [11, 3], [17, 5], [13, 4], [18, 6], [7, 6], [5, 4], 
                                   [5, 6]], 
                                  [[5, 6], [4, 2], [14, 4], 
                                [9, 3], [8, 3], [15, 4], [10, 3], [15, 4], [11, 3], [19, 5]], [[10, 4], [4, 3], 
                                [11, 6], [16, 4], [6, 2], [6, 2], [18, 5], [10, 6], [17, 5], [4, 3]], [[9, 3], 
                                [14, 4], [6, 4], [19, 6], [4, 2], [11, 3], [19, 6], [8, 2], [9, 6], [5, 3]], 
                                [[10, 3], [7, 5], [19, 5], [15, 6], [12, 5], [16, 4], [19, 5], [12, 5], [15, 5], 
                                [18, 5]], [[16, 4], [13, 4], [12, 5], [11, 3], [14, 4], [7, 5], [11, 3], [15, 4], 
                                 [13, 4], [5, 3]], 
                                [[8, 2], [10, 3], [13, 4], [20, 5], [17, 5], [6, 2], [7, 2], [14, 4], [13, 4], 
                                 [7, 5]], [[18, 5], [8, 2], [17, 5], [14, 4], [10, 6], [17, 5], [6, 5], [6, 5], 
                                           [4, 5], [7, 4]], 
                                  [[13, 6], [6, 5], [11, 6], [10, 3], [6, 4], [7, 2], [17, 6], [12, 3], [5, 2], 
                                   [16, 5]], [[10, 3], [19, 6], [8, 5], [12, 3], [11, 5], [19, 5], [9, 3], 
                                              [17, 5], [19, 5], [3, 4]]] )
    

#     normal_parameters = np.array([
#         [[14,  4], [ 5,  1], [11,  1], [15,  2], [18,  2], [19 , 3], [12 , 4], [ 7 , 1], [ 9,  2], [12,  4],
#          [18 , 1], [18  ,3], [ 9,  3], [17 , 4], [ 7  ,2], [16,  5], [17,  5], [11 , 2], [12 , 2], [14 , 3],
#          [17 , 1], [16,  3], [ 8,  2], [11,  3], [ 5,  1], [18 , 2], [20 , 5], [10 , 3], [ 6 , 2], [10 , 3]],
        
#         [[ 5 , 1], [11 , 3], [ 6,  2], [ 6,  2], [ 7 , 1], [ 7,  2], [13,  4], [15 , 2], [ 5,  1], [19,  1],
#          [16 , 1], [ 7  ,2], [ 4 , 1], [10 , 2], [ 7  ,2], [ 8,  2], [14 , 1], [17 , 5], [19 , 5], [14 , 3],
#          [20 , 1], [12,  3], [13 , 4], [16 , 3], [ 7  ,2], [14 , 3], [17 , 1], [15 , 4], [ 6 , 2], [13 , 4]],
        
#         [[16,  3], [20 , 2], [20 , 2], [17 , 2], [14 , 4], [18 , 3], [18 , 1], [10  ,3], [11 , 2], [19 , 5],
#          [ 9,  3], [19 , 4], [15 , 5], [10 , 2], [ 5 , 1], [ 4 , 1], [ 5 , 1], [ 5 , 1], [ 5 , 1], [20 , 6],
#          [15,  1], [ 5 , 1], [10 , 3], [20 , 2], [15 , 5], [ 3 , 1], [ 8 , 2], [13 , 4], [20 , 4], [ 8 , 2]], 
        
#         [[10 , 3], [ 9  ,3], [ 9 , 3], [19 , 2], [ 9 , 3], [ 8 , 2], [15 , 2], [11 , 3], [12 , 2], [ 6 , 2],
#          [14 , 1], [20 , 6], [12 , 1], [ 6 , 2], [18 , 4], [ 6 , 2], [12 , 2], [ 9 , 3], [ 9 , 3], [ 7 , 2],
#          [10 , 3], [13 , 4], [20 , 5], [16 , 5], [ 4 , 1], [19 , 2], [ 9 , 3], [ 9 , 3], [ 6 , 2], [12 , 1]],
        
#         [[11 , 2], [ 6 , 2], [14 , 1], [13 , 1], [10 , 2], [ 3 , 1], [20 , 2], [12 , 1], [ 8 , 2], [16 , 4],
#          [ 4 , 1], [ 4 , 1], [ 3 , 1], [ 5 , 1], [19 , 2], [ 7 , 2], [10 , 3], [15 , 1], [ 8 , 1], [12 , 1],
#          [12 , 4], [20 , 6], [16 , 5], [ 9 , 3], [11 , 3], [11 , 3], [17 , 5], [ 9 , 3], [ 3 , 1], [ 9  ,3]],
        
#         [[ 5 , 1], [ 3 , 1], [ 9 , 2], [16 , 3], [11 , 3], [14  ,4], [ 6 , 2], [ 6 , 2], [ 5 , 1], [19 , 5],
#          [ 7 , 2], [11 , 1], [ 3 , 1], [13 , 4], [10 , 3], [16 , 5], [14 , 3], [12 , 4], [ 9 , 3], [ 5 , 1],
#          [19 , 4], [ 5 , 1], [16 , 3], [11 , 3], [11 , 2], [12 , 2], [ 6 , 2], [15 , 2], [15 , 5], [13 , 1]],
                  
#         [[ 6 , 2], [19 , 3], [14 , 1], [ 7 , 2], [19 , 6], [ 5 , 1], [12 , 3], [ 4 , 1], [15 , 5], [ 9 , 3],
#          [ 6 , 2], [19 , 3], [16 , 4], [11 , 3], [ 8 , 2], [14 , 3], [12 , 4], [ 7 , 2], [17 , 1], [ 3 , 1],
#          [17 , 4], [20 , 4], [15 , 3], [13 , 4], [ 8 , 2], [11 , 3], [20 , 3], [20 , 1], [17 , 1], [11 , 3]],
                  
#         [[ 8 , 1], [11 , 2], [ 4 , 1], [14 , 3], [11 , 2], [ 5 , 1], [ 9 , 3], [ 3 , 1], [ 6 , 2], [ 7 , 2],
#          [10 , 2], [ 6 , 2], [15 , 3], [15 , 2], [18 , 1], [14 , 2], [ 9 , 3], [11 , 3], [ 3 , 1], [ 3  ,1],
#          [ 7 , 2], [ 9 , 2], [16 , 3], [20 , 3], [15 , 5], [ 9 , 3], [13 , 3], [ 3 , 1], [15 , 5], [12 , 4]],
                  
#         [[18 , 1], [16 , 3], [ 4 , 1], [ 3 , 1], [ 9 , 3], [ 5 , 1], [10 , 3], [ 3 , 1], [12 , 2], [14 , 4],
#          [13 , 2], [ 3 , 1], [ 4 , 1], [13 , 4], [20 , 2], [18 , 3], [12 , 4], [17 , 3], [19 , 4], [15 , 1],
#          [13 , 4], [ 7 , 1], [ 3 , 1], [14 , 3], [19 , 4], [ 9 , 2], [ 3 , 1], [14 , 2], [19 , 6], [14 , 3]], 
                  
#         [[ 7 , 2], [ 5 , 1], [ 5 , 1], [13 , 2], [ 3 , 1], [16 , 4], [ 8 , 2], [ 5 , 1], [13 , 4], [16 , 1],
#          [15 , 4], [ 6 , 2], [13 , 2], [13 , 4], [ 9 , 3], [13 , 4], [ 5 , 1], [17 , 5], [ 5 , 1], [14 , 4],
#          [11 , 1], [ 6 , 1], [ 3 , 1], [12 , 3], [18 , 3], [ 4 , 1], [15 , 2], [19 , 2], [16 , 3], [ 7 , 1]]])

    
    print("\n")
    print("PCS:", CS, ", tau:", tau, ", H:", H, ", Budget:",Budget)
    
#     Initialize an allocation 'A'
    choice = 1
    for c in range(choice):
        # print('--------------')
        # print("Policy:",c+1)
        initial_A = initialAllocation(c, normal_parameters)
        # print("Initial Allocation:", initial_A, 'sum(initial_A):', sum(initial_A))
        final_A = greedy_distribution(initial_A, normal_parameters)
#         print("Final Allocation:", final_A)

In [14]:
if __name__ == "__main__":
    main()

********Greedy + MonteCarlo********


PCS: 15 , tau: 3 , H: 10 , Budget: 50


C:\Users\admin\anaconda3\Lib\fractions.py:754: RuntimeWarning: overflow encountered in scalar multiply
  return Fraction._from_coprime_ints(na * nb, db * da)
C:\Users\admin\anaconda3\Lib\fractions.py:732: RuntimeWarning: overflow encountered in scalar subtract
  return Fraction._from_coprime_ints(na * db - da * nb, da * db)


AttributeError: 'numpy.int32' object has no attribute 'bit_length'